In [119]:
#Instalando o SDK do Google
!pip install -q -U google-generativeai

Configurações

In [6]:
#Importações e configurações iniciais
import numpy as np
import pandas as pd
import random
import google.generativeai as genai
from google.colab import userdata
from IPython.display import clear_output

# configurar api_key
api_key = userdata.get("SECRET_KEY")
genai.configure(api_key=api_key )

In [7]:
# configuração de resposta precisa ou criativa(criativa(0)< valor<precisa(1))
generation_config = {
  "temperature": 0,
  "candidate_count": 1
}

In [8]:
safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
    }

In [9]:
# configurando modelo
model = "models/embedding-001"

In [10]:
# função para capturar o quanto o dado tem haver com numeros
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [11]:
#Listagem de documentos que serão buscados
documentos = [
    {
    "palavra": "ありがとう",
    "traducao": "Obrigado"
    },

   {
    "palavra": "はじめまして",
     "traducao": "Praze em conhece-lo"
     },

   {
    "palavra": "行く",
     "traducao": "Ir"
     }
]


In [12]:
# transformando em um dataFrame e editando nome coluna
df = pd.DataFrame(documentos)
df.columns = ["palavra",  "traducao"]

In [13]:
# Adicionando os embedding no dataFrame(linha por linha)
df["Embeddings"] = df.apply(lambda row: embed_fn(row["palavra"], row[ "traducao"]), axis=1)

In [14]:
# gera a resposta com base no embedding da consulta com a base
def gerar_e_buscar_consulta(consulta, base, model):

  embedding_da_consulta = genai.embed_content(model=model,
                                              content=consulta,
                                              task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice][ "traducao"]

In [15]:
modelGenerative = genai.GenerativeModel(model_name='gemini-1.0-pro',generation_config=generation_config,safety_settings=safety_settings)

In [41]:
def mensagemMenu():
  print("""
Digite 1 para armazenar a palavra e sua tradução
Digite 2 para formar frase
Digite 3 traduza a frase apresentada

Obs: para finalizar digite '0'
""")


In [ ]:
chat = modelGenerative.start_chat(history=[])

print("""*-------------------*
| Seja Bem vindo!!! |
*-------------------*""")
mensagemMenu()

prompt = int(input('Esperando prompt: '))
while prompt !=0:

      if prompt==1:
        print("""
        *--------------------------------------------*
        | Digite a palavra(ex: arigatou - Obrigado)  |
        | Obs: para finalizar digite fim             |
        *--------------------------------------------*
        """)
        word = input('Digite a palavra: ').split("-")

        # Adiciona palavras na lista
        while word[0] != "fim":
          documentos.append({"palavra":word[0].strip(),"traducao":word[1].strip()})
          word = input('Digite a palavra: ').split("-")

        clear_output()
      if prompt==2:
        if len(documentos)>1:

           # lista de palavra
           palavras = list(map(lambda doc: doc["palavra"], documentos))
           promptAdapted = f"Escreva uma frase com uma ou mais palavras dessa lista em uma frase simples do cotidiano para uma iniciante da lingua Japonesa que termine com um ponto final e sua estrutura com tradução, caso tenha kanji coloque seu furigana: {palavras}"

           # gera uma frase com as palavras
           response = chat.send_message(promptAdapted)
           print("----------"*len(response.text))
           print("Resposta:", response.text, '\n')
           print("----------"*len(response.text))
        else:
          print("Não existe palavra armazenada")
      if prompt==3:
        # Seleciona aleatoriamente um objeto da lista
        documento_aleatorio = random.choice(documentos)

        # Obtém a palavra do objeto selecionado
        palavra_aleatoria = documento_aleatorio["palavra"]
        traducaoArmazenada = documento_aleatorio["traducao"]

        print(f"A palavra selecionada é: {palavra_aleatoria}")
        traducao = input('Digite a tradução da palavra: ')

        # valida se a palavra é igual ou se é parecida
        promptAdapted = f"Essas palavras são parecidas {traducao} com {traducaoArmazenada}?(sim/não)"
        response = chat.send_message(promptAdapted)

        #caso for parecida ou igual a tradução
        if response.text =="sim":
          response = chat.send_message(f"Exiba uma mensagem parabenizando um estudade de japones com {traducaoArmazenada}")
         #caso for errado
        else:
          print(f"\nA tradução da palavra é: {traducaoArmazenada}")
          response = chat.send_message(f"Exiba  uma  mensagem de incentivo")

        print(f"""{"-"*len(response.text)}
{response.text}
{"-"*len(response.text)}""")
      mensagemMenu()
      prompt = int(input('Esperando prompt: '))

      #limpa o console da saida da celula
      clear_output()

*-------------------*
| Seja Bem vindo!!! |
*-------------------*

Digite 1 para armazenar a palavra e sua tradução
Digite 2 para formar frase
Digite 3 traduza a frase apresentada

Obs: para finalizar digite '0'  

Esperando prompt: 2
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
print(chat.history)